# Análise de Absentismo - Apresentação à Direção

## Abordagem: Storytelling com Dados

Este notebook responde a perguntas específicas sobre absentismo, construindo um argumento
baseado em evidências para apresentação à direção.

### Estrutura:
1. **Panorama Geral** - Qual o contexto?
2. **Padrões Temporais** - Há sazonalidade?
3. **Teste de Aleatoriedade** - Faltas são aleatórias ou estratégicas?
4. **Análise de Atrasos** - Mesmos padrões?
5. **Segmentação** - Varia por categoria/operação?
6. **Red Flags** - Há abusos óbvios?
7. **Conclusões** - O que fazemos?

---

## Setup e Configuração

In [ ]:
# Imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import chisquare, chi2_contingency
from datetime import datetime
import warnings

# Configurações
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Cores para consistência visual
COLORS = {
    'primary': '#2E86AB',
    'success': '#06A77D',
    'warning': '#F18F01',
    'danger': '#C73E1D',
    'info': '#6A994E'
}

print('✅ Setup concluído!')

## Carregamento e Preparação dos Dados

In [ ]:
# Carregar códigos com nova estrutura (6 colunas)
codigos = pd.read_excel('códigos.xlsx', engine='openpyxl')
codigos.columns = ['codigo', 'descricao', 'nivel_1', 'nivel_2', 'nivel_3', 'nivel_4']

# Limpar espaços
for col in codigos.columns:
    if codigos[col].dtype == 'object':
        codigos[col] = codigos[col].str.strip()

print(f'✅ Códigos carregados: {len(codigos)} códigos')
print(f'   Níveis de classificação:')
print(f'   - Nível 1: {codigos["nivel_1"].nunique()} categorias (Presença/Ausência)')
print(f'   - Nível 2: {codigos["nivel_2"].nunique()} categorias')
print(f'   - Nível 3: {codigos["nivel_3"].nunique()} categorias')
print(f'   - Nível 4: {codigos["nivel_4"].nunique()} categorias (inclui Atrasos)')

In [ ]:
# Carregar dataset
print('Carregando dataset...')

dtype_dict = {
    'login_colaborador': 'int32',
    'nome_colaborador': 'str',
    'categoria_profissional': 'category',
    'segmento': 'category',
    'operacao': 'category',
    'departamento': 'category',
    'segmento_processado_codigo': 'category',
    'codigo_nivel_2': 'category',
    'Contagem_ID': 'int8',
    'Activo?': 'category'
}

df_raw = pd.read_csv('combined_data.csv', sep=',', dtype=dtype_dict, parse_dates=['Data'])

# Merge com códigos (agora com 4 níveis)
df_raw = df_raw.merge(codigos[['codigo', 'nivel_1', 'nivel_2', 'nivel_3', 'nivel_4']], 
                       left_on='segmento_processado_codigo', right_on='codigo', how='left')
df_raw.drop('codigo', axis=1, inplace=True)

# Variáveis derivadas
df_raw['ano'] = df_raw['Data'].dt.year
df_raw['mes'] = df_raw['Data'].dt.month
df_raw['dia'] = df_raw['Data'].dt.day
df_raw['dia_semana'] = df_raw['Data'].dt.dayofweek
df_raw['nome_dia_semana'] = df_raw['Data'].dt.day_name()
df_raw['ano_mes'] = df_raw['Data'].dt.to_period('M')
df_raw['semana_mes'] = ((df_raw['dia'] - 1) // 7) + 1
df_raw['is_ativo'] = (df_raw['Activo?'] == 'Sim').astype(int)

print(f'✅ Dataset carregado: {len(df_raw):,} registros')

In [ ]:
# CRÍTICO: Agregar dados (1 registro por colaborador por dia)
# Hierarquia de prioridade baseada em nivel_2 (mantém apenas o mais grave)
# Ordem: Ausência Injustificada > Ausência Justificada > Atraso > Presença > Inatividade

prioridade_nivel2 = {
    'Ausência Injustificada': 1,
    'Ausência Justificada': 2,
    'Presença': 3,  # Nota: Presença inclui atrasos no nivel_2
    'Inatividade': 4
}

df_raw['prioridade'] = df_raw['nivel_2'].map(prioridade_nivel2)

# Para desempatar dentro de "Presença", usar nivel_4 (Atraso tem prioridade sobre Presença normal)
prioridade_nivel4 = {
    'Atraso': 1,
    'Presença': 2,
    'Ferias / Feriado / Folga': 3,
    'Ausência Médica': 4,
    'Licença Mat / Pat': 5,
    'Ausência Justificada': 6,
    'Ausência Injustificada': 7,
    'Assistência Familiar': 8,
    'Óbito': 9,
    'Casamento': 10,
    'Exame Escolar': 11,
    'Inatividade': 12
}

df_raw['prioridade_nivel4'] = df_raw['nivel_4'].map(prioridade_nivel4)

# Ordenar por prioridade (primeiro nivel_2, depois nivel_4 para desempate)
df = df_raw.sort_values(['prioridade', 'prioridade_nivel4']).groupby(['login_colaborador', 'Data'], as_index=False).first()
df.drop(['prioridade', 'prioridade_nivel4'], axis=1, inplace=True)

print(f'✅ Dataset agregado: {len(df):,} dias-colaborador (1 registro por pessoa por dia)')
print(f'   Redução: {len(df_raw) - len(df):,} registros duplicados removidos')
print(f'\nDistribuição por Nível 1:')
print(df['nivel_1'].value_counts())
print(f'\nDistribuição por Nível 2:')
print(df['nivel_2'].value_counts())

---

# SEÇÃO 1: Panorama Geral

## Pergunta 1: Quantos dias de trabalho estamos a analisar?

In [ ]:
# Estatísticas gerais
total_dias_colab = len(df)
periodo_inicio = df['Data'].min()
periodo_fim = df['Data'].max()
dias_calendario = (periodo_fim - periodo_inicio).days + 1
total_colaboradores = df['login_colaborador'].nunique()
colaboradores_ativos = df[df['is_ativo']==1]['login_colaborador'].nunique()

print('='*80)
print('📊 PANORAMA GERAL')
print('='*80)
print(f'\nPeríodo Analisado: {periodo_inicio.strftime("%d/%m/%Y")} a {periodo_fim.strftime("%d/%m/%Y")}')
print(f'Dias no calendário: {dias_calendario} dias')
print(f'\nTotal de dias-colaborador analisados: {total_dias_colab:,}')
print(f'  (1 dia-colaborador = 1 pessoa trabalhando 1 dia)')
print(f'\nColaboradores únicos: {total_colaboradores:,}')
print(f'  • Ativos: {colaboradores_ativos:,} ({colaboradores_ativos/total_colaboradores*100:.1f}%)')
print(f'  • Inativos: {total_colaboradores - colaboradores_ativos:,}')
print('='*80)

## Pergunta 2: Visão Hierárquica - Presença vs Ausência (Nível 1)

**Importante**: No Nível 1, os Atrasos são contabilizados como "Presença" (colaborador estava presente, embora atrasado).

In [ ]:
# Distribuição Nível 1 (Presença vs Ausência)
dist_nivel1 = df['nivel_1'].value_counts()
dist_nivel1_pct = (dist_nivel1 / len(df) * 100).round(2)

# Gráfico pizza para Nível 1
fig = go.Figure(data=[go.Pie(
    labels=dist_nivel1.index,
    values=dist_nivel1.values,
    marker=dict(colors=[COLORS['success'], COLORS['danger']]),
    hole=0.5,
    textinfo='label+percent',
    textposition='outside',
    textfont=dict(size=14)
)])

fig.update_layout(
    title='<b>Nível 1: Presença vs Ausência</b><br><sub>(Atrasos contam como Presença)</sub>',
    height=450,
    showlegend=True,
    annotations=[dict(text=f'{len(df):,}<br>dias-colab', x=0.5, y=0.5, font_size=16, showarrow=False)]
)

fig.show()

# Tabela resumo
print('\n📊 Resumo Nível 1:')
for categoria in dist_nivel1.index:
    count = dist_nivel1[categoria]
    pct = dist_nivel1_pct[categoria]
    print(f'  • {categoria}: {count:,} dias ({pct:.2f}%)')

print(f'\n💡 Taxa de Absentismo (Nível 1): {dist_nivel1_pct.get("Ausência", 0):.2f}%')

## Pergunta 3: Drill-down por Níveis (2, 3, e 4)

Agora vamos detalhar cada nível de classificação para entender melhor a composição do absentismo.

In [ ]:
# Criar subplots para mostrar os 3 níveis lado a lado
from plotly.subplots import make_subplots

# Distribuições
dist_nivel2 = df['nivel_2'].value_counts().sort_values(ascending=True)
dist_nivel3 = df['nivel_3'].value_counts().sort_values(ascending=False).head(10)
dist_nivel4 = df['nivel_4'].value_counts().sort_values(ascending=False).head(10)

# Criar figura com 3 subplots
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Nível 2 (4 categorias)', 'Nível 3 - Top 10', 'Nível 4 - Top 10'),
    specs=[[{"type": "bar"}, {"type": "bar"}, {"type": "bar"}]]
)

# Nível 2
fig.add_trace(
    go.Bar(
        y=dist_nivel2.index,
        x=dist_nivel2.values,
        orientation='h',
        marker=dict(color=COLORS['primary']),
        text=[f"{v:,}<br>({v/len(df)*100:.1f}%)" for v in dist_nivel2.values],
        textposition='outside',
        showlegend=False
    ),
    row=1, col=1
)

# Nível 3
fig.add_trace(
    go.Bar(
        y=dist_nivel3.index,
        x=dist_nivel3.values,
        orientation='h',
        marker=dict(color=COLORS['info']),
        text=[f"{v:,}<br>({v/len(df)*100:.1f}%)" for v in dist_nivel3.values],
        textposition='outside',
        showlegend=False
    ),
    row=1, col=2
)

# Nível 4
fig.add_trace(
    go.Bar(
        y=dist_nivel4.index,
        x=dist_nivel4.values,
        orientation='h',
        marker=dict(color=COLORS['warning']),
        text=[f"{v:,}<br>({v/len(df)*100:.1f}%)" for v in dist_nivel4.values],
        textposition='outside',
        showlegend=False
    ),
    row=1, col=3
)

fig.update_xaxes(title_text="Dias", row=1, col=1)
fig.update_xaxes(title_text="Dias", row=1, col=2)
fig.update_xaxes(title_text="Dias", row=1, col=3)

fig.update_layout(
    height=500,
    showlegend=False,
    title_text="<b>Distribuição por Níveis de Classificação</b>"
)

fig.show()

# Tabelas resumo
print('\n📋 NÍVEL 2 (4 categorias):')
for cat in dist_nivel2.index[::-1]:
    count = dist_nivel2[cat]
    pct = count / len(df) * 100
    print(f'  • {cat}: {count:,} ({pct:.2f}%)')

print('\n📋 NÍVEL 4 - Top 10 (inclui Atrasos separados):')
for cat in dist_nivel4.index:
    count = dist_nivel4[cat]
    pct = count / len(df) * 100
    simbolo = '⚠️' if cat == 'Atraso' else '🔴' if 'Injustificada' in cat else '🟡' if 'Médica' in cat or 'Licença' in cat else '🟢'
    print(f'  {simbolo} {cat}: {count:,} ({pct:.2f}%)')

---

# SEÇÃO 2: Padrões Temporais e Sazonalidade

## Pergunta 4: Há diferenças por mês? Existe sazonalidade?

In [ ]:
# Filtrar ausências e atrasos (excluir apenas presenças normais e inatividade)
# Usando nivel_1 = 'Ausência' OU nivel_4 = 'Atraso'
df_problemas = df[(df['nivel_1'] == 'Ausência') | (df['nivel_4'] == 'Atraso')].copy()

# Evolução mensal usando nivel_2 para categorização principal
evolucao_mensal = df_problemas.groupby(['ano_mes', 'nivel_2']).size().unstack(fill_value=0)
evolucao_mensal['Total'] = evolucao_mensal.sum(axis=1)

# Gráfico de área empilhada
fig = go.Figure()

# Cores por categoria nivel_2
cores_nivel2 = {
    'Ausência Injustificada': COLORS['danger'],
    'Ausência Justificada': COLORS['info'],
    'Presença': COLORS['warning'],  # Atrasos aparecem como Presença no nivel_2
    'Inatividade': '#D3D3D3'
}

for cat in evolucao_mensal.columns:
    if cat != 'Total' and cat in cores_nivel2:
        fig.add_trace(go.Scatter(
            x=evolucao_mensal.index.astype(str),
            y=evolucao_mensal[cat],
            name=cat + (' (inclui Atrasos)' if cat == 'Presença' else ''),
            mode='lines+markers',
            stackgroup='one',
            marker=dict(color=cores_nivel2[cat]),
            line=dict(width=2)
        ))

fig.update_layout(
    title='Evolução Mensal de Ausências e Atrasos',
    xaxis_title='Mês',
    yaxis_title='Número de Dias',
    height=500,
    hovermode='x unified'
)

fig.show()

# Estatísticas
print('\n📊 Estatísticas Mensais:')
print(f'  Média mensal: {evolucao_mensal["Total"].mean():.0f} dias')
print(f'  Mês com mais problemas: {evolucao_mensal["Total"].idxmax()} ({evolucao_mensal["Total"].max()} dias)')
print(f'  Mês com menos problemas: {evolucao_mensal["Total"].idxmin()} ({evolucao_mensal["Total"].min()} dias)')
print(f'  Variação: {(evolucao_mensal["Total"].max() - evolucao_mensal["Total"].min()) / evolucao_mensal["Total"].mean() * 100:.1f}% vs média')

print(f'\n💡 Total de dias-problema no período: {len(df_problemas):,} ({len(df_problemas)/len(df)*100:.2f}% dos dias)')

### Tendência: Está a melhorar ou piorar ao longo do tempo?

In [ ]:
# Calcular tendência usando média móvel
evolucao_mensal['Media_Movel_3M'] = evolucao_mensal['Total'].rolling(window=3, center=True).mean()

# Gráfico com tendência
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=evolucao_mensal.index.astype(str),
    y=evolucao_mensal['Total'],
    name='Total Mensal',
    mode='lines+markers',
    marker=dict(size=8, color=COLORS['primary']),
    line=dict(width=2)
))

fig.add_trace(go.Scatter(
    x=evolucao_mensal.index.astype(str),
    y=evolucao_mensal['Media_Movel_3M'],
    name='Tendência (Média Móvel 3M)',
    mode='lines',
    line=dict(width=3, dash='dash', color=COLORS['danger'])
))

fig.update_layout(
    title='Tendência de Absentismo ao Longo do Tempo',
    xaxis_title='Mês',
    yaxis_title='Número de Dias',
    height=450
)

fig.show()

# Comparar primeiro vs último trimestre
primeiro_trim = evolucao_mensal['Total'].iloc[:3].mean()
ultimo_trim = evolucao_mensal['Total'].iloc[-3:].mean()
variacao_pct = (ultimo_trim - primeiro_trim) / primeiro_trim * 100

print(f'\n📈 Análise de Tendência:')
print(f'  Média primeiro trimestre: {primeiro_trim:.0f} dias/mês')
print(f'  Média último trimestre: {ultimo_trim:.0f} dias/mês')
if variacao_pct > 0:
    print(f'  🔴 PIOROU: +{variacao_pct:.1f}% vs início do período')
else:
    print(f'  🟢 MELHOROU: {variacao_pct:.1f}% vs início do período')

## Pergunta 5: Há padrões por dia da semana?

In [ ]:
# Distribuição por dia da semana
dias_ordem = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dias_pt = ['Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta', 'Sábado', 'Domingo']

dist_dia_semana = df_problemas['nome_dia_semana'].value_counts().reindex(dias_ordem)

# Gráfico de barras
fig = go.Figure(data=[go.Bar(
    x=dias_pt,
    y=dist_dia_semana.values,
    marker=dict(color=[
        COLORS['danger'] if dia in ['Monday', 'Friday'] else COLORS['primary'] 
        for dia in dias_ordem
    ]),
    text=[f"{v:,}" for v in dist_dia_semana.values],
    textposition='outside'
)])

# Adicionar linha de média
media = dist_dia_semana.mean()
fig.add_hline(y=media, line_dash='dash', line_color='red', 
              annotation_text=f'Média: {media:.0f}',
              annotation_position='right')

fig.update_layout(
    title='Distribuição de Ausências e Atrasos por Dia da Semana',
    xaxis_title='Dia da Semana',
    yaxis_title='Número de Ocorrências',
    height=450
)

fig.show()

# Estatísticas
print('\n📊 Análise por Dia da Semana:')
for dia_en, dia_pt, count in zip(dias_ordem, dias_pt, dist_dia_semana.values):
    pct = count / dist_dia_semana.sum() * 100
    diff_media = (count - media) / media * 100
    simbolo = '🔴' if diff_media > 10 else '🟢' if diff_media < -10 else '⚪'
    print(f'  {simbolo} {dia_pt}: {count:,} ({pct:.1f}%) - {diff_media:+.1f}% vs média')

### Heatmap: Dia da Semana vs Semana do Mês

In [ ]:
# Criar heatmap
heatmap_data = df_problemas.groupby(['dia_semana', 'semana_mes']).size().unstack(fill_value=0)
heatmap_data.index = heatmap_data.index.map(dict(zip(range(7), dias_pt)))

fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,
    x=[f'Semana {i}' for i in heatmap_data.columns],
    y=heatmap_data.index,
    colorscale='Reds',
    text=heatmap_data.values,
    texttemplate='%{text}',
    textfont={"size":10},
    colorbar=dict(title='Ocorrências')
))

fig.update_layout(
    title='Heatmap: Padrão de Faltas/Atrasos por Dia da Semana e Semana do Mês',
    xaxis_title='Semana do Mês',
    yaxis_title='Dia da Semana',
    height=400
)

fig.show()

---

# SEÇÃO 3: Teste de Aleatoriedade

## Pergunta 6: Se as faltas fossem aleatórias, a distribuição seria uniforme?

**Hipótese**: Se colaboradores faltassem de forma aleatória (sem preferência por dias específicos),
esperaríamos que a distribuição de faltas por dia da semana fosse proporcional ao número de
colaboradores que trabalham em cada dia.

Vamos testar esta hipótese estatisticamente.

In [ ]:
# Contar dias de trabalho disponíveis por dia da semana (total de oportunidades)
# Usando TODOS os registros (não só problemas)
oportunidades_por_dia = df['nome_dia_semana'].value_counts().reindex(dias_ordem)

# Contar faltas/atrasos observados por dia da semana
observado = df_problemas['nome_dia_semana'].value_counts().reindex(dias_ordem)

# Calcular o esperado se fosse aleatório (proporcional às oportunidades)
proporcao_esperada = oportunidades_por_dia / oportunidades_por_dia.sum()
total_problemas = observado.sum()
esperado = proporcao_esperada * total_problemas

# Teste Chi-quadrado
from scipy.stats import chisquare
chi2_stat, p_value = chisquare(f_obs=observado, f_exp=esperado)

# Criar tabela comparativa
comparacao = pd.DataFrame({
    'Dia': dias_pt,
    'Observado': observado.values,
    'Esperado (Aleatório)': esperado.values.round(0).astype(int),
    'Diferença': (observado.values - esperado.values).round(0).astype(int),
    'Diferença %': ((observado.values - esperado.values) / esperado.values * 100).round(1)
})

print('='*90)
print('🎲 TESTE DE ALEATORIEDADE - DIA DA SEMANA')
print('='*90)
print('\nHipótese Nula (H0): Faltas são distribuídas aleatoriamente por dia da semana')
print(f'\nResultado do Teste Chi-Quadrado:')
print(f'  χ² = {chi2_stat:.2f}')
print(f'  p-value = {p_value:.6f}')

if p_value < 0.001:
    print(f'\n🔴 CONCLUSÃO: Rejeitamos H0 com alta confiança (p < 0.001)')
    print(f'   As faltas NÃO são aleatórias - há padrões claros!')
else:
    print(f'\n🟢 CONCLUSÃO: Não podemos rejeitar H0')
    print(f'   As faltas parecem ser aleatórias')

print('\n' + '='*90)
print('\nTabela Comparativa:')
print(comparacao.to_string(index=False))
print('='*90)

In [ ]:
# Visualização: Observado vs Esperado
fig = go.Figure()

fig.add_trace(go.Bar(
    x=dias_pt,
    y=observado.values,
    name='Observado (Real)',
    marker=dict(color=COLORS['danger']),
    text=[f"{v:,}" for v in observado.values],
    textposition='outside'
))

fig.add_trace(go.Bar(
    x=dias_pt,
    y=esperado.values,
    name='Esperado (Se Aleatório)',
    marker=dict(color=COLORS['info'], opacity=0.6),
    text=[f"{v:.0f}" for v in esperado.values],
    textposition='outside'
))

fig.update_layout(
    title='Teste de Aleatoriedade: Observado vs Esperado',
    xaxis_title='Dia da Semana',
    yaxis_title='Número de Faltas/Atrasos',
    barmode='group',
    height=500
)

fig.show()

## Pergunta 7: Há concentração de faltas no início ou fim do mês?

In [ ]:
# Categorizar por período do mês
def categorizar_periodo(dia):
    if dia <= 7:
        return 'Início (1-7)'
    elif dia <= 15:
        return 'Meio-Início (8-15)'
    elif dia <= 22:
        return 'Meio-Fim (16-22)'
    else:
        return 'Fim (23+)'

df_problemas['periodo_mes'] = df_problemas['dia'].apply(categorizar_periodo)

# Contar observado
periodos_ordem = ['Início (1-7)', 'Meio-Início (8-15)', 'Meio-Fim (16-22)', 'Fim (23+)']
observado_periodo = df_problemas['periodo_mes'].value_counts().reindex(periodos_ordem)

# Calcular esperado (se aleatório, seria proporcional aos dias disponíveis)
# Início: 7 dias, Meio-Início: 8 dias, Meio-Fim: 7 dias, Fim: ~8 dias (média)
dias_por_periodo = [7, 8, 7, 8]  # aproximação
proporcao_periodo = np.array(dias_por_periodo) / sum(dias_por_periodo)
esperado_periodo = proporcao_periodo * observado_periodo.sum()

# Teste Chi-quadrado
chi2_periodo, p_value_periodo = chisquare(f_obs=observado_periodo, f_exp=esperado_periodo)

print('='*90)
print('🎲 TESTE DE ALEATORIEDADE - PERÍODO DO MÊS')
print('='*90)
print(f'\nResultado do Teste Chi-Quadrado:')
print(f'  χ² = {chi2_periodo:.2f}')
print(f'  p-value = {p_value_periodo:.6f}')

if p_value_periodo < 0.05:
    print(f'\n🔴 CONCLUSÃO: Rejeitamos H0 (p < 0.05)')
    print(f'   Há padrão significativo ao longo do mês!')
else:
    print(f'\n🟢 CONCLUSÃO: Distribuição compatível com aleatoriedade')

# Tabela
comparacao_periodo = pd.DataFrame({
    'Período': periodos_ordem,
    'Observado': observado_periodo.values,
    'Esperado': esperado_periodo.round(0).astype(int),
    'Diferença %': ((observado_periodo.values - esperado_periodo) / esperado_periodo * 100).round(1)
})

print('\n' + comparacao_periodo.to_string(index=False))
print('='*90)

# Gráfico
fig = go.Figure()

fig.add_trace(go.Bar(
    x=periodos_ordem,
    y=observado_periodo.values,
    name='Observado',
    marker=dict(color=COLORS['warning'])
))

fig.add_trace(go.Bar(
    x=periodos_ordem,
    y=esperado_periodo,
    name='Esperado (Aleatório)',
    marker=dict(color=COLORS['info'], opacity=0.6)
))

fig.update_layout(
    title='Distribuição de Faltas por Período do Mês',
    xaxis_title='Período',
    yaxis_title='Número de Ocorrências',
    barmode='group',
    height=450
)

fig.show()

---

# SEÇÃO 4: Análise de Atrasos

## Pergunta 8: Os atrasos seguem os mesmos padrões que as faltas?

In [ ]:
# Filtrar apenas atrasos (agora disponível em nivel_4)
df_atrasos = df[df['nivel_4'] == 'Atraso'].copy()

print(f'Total de dias com atrasos: {len(df_atrasos):,}')
print(f'Percentagem do total de dias: {len(df_atrasos)/len(df)*100:.2f}%')
print(f'Percentagem dos dias-problema (ausências + atrasos): {len(df_atrasos)/len(df_problemas)*100:.1f}%')

# Distribuição por dia da semana
atrasos_dia = df_atrasos['nome_dia_semana'].value_counts().reindex(dias_ordem)

fig = go.Figure(data=[go.Bar(
    x=dias_pt,
    y=atrasos_dia.values,
    marker=dict(color=COLORS['warning']),
    text=[f"{v:,}" for v in atrasos_dia.values],
    textposition='outside'
)])

media_atrasos = atrasos_dia.mean()
fig.add_hline(y=media_atrasos, line_dash='dash', line_color='red',
              annotation_text=f'Média: {media_atrasos:.0f}')

fig.update_layout(
    title='Distribuição de Atrasos por Dia da Semana',
    xaxis_title='Dia',
    yaxis_title='Número de Atrasos',
    height=450
)

fig.show()

# Análise
print('\n📊 Padrão de Atrasos:')
for dia_pt, count in zip(dias_pt, atrasos_dia.values):
    diff = (count - media_atrasos) / media_atrasos * 100
    simbolo = '🔴' if diff > 10 else '🟢' if diff < -10 else '⚪'
    print(f'  {simbolo} {dia_pt}: {count:,} ({diff:+.1f}% vs média)')

---

# SEÇÃO 5: Segmentação

## Pergunta 9: Os padrões variam por Categoria Profissional ou Operação?

In [ ]:
# Taxa de absentismo por categoria profissional (Top 10)
top_categorias = df['categoria_profissional'].value_counts().head(10).index
df_top_cat = df[df['categoria_profissional'].isin(top_categorias)]

# Calcular taxa (usando df_problemas: ausências + atrasos)
problemas_cat = df_top_cat[(df_top_cat['nivel_1'] == 'Ausência') | (df_top_cat['nivel_4'] == 'Atraso')].groupby('categoria_profissional').size()
total_cat = df_top_cat.groupby('categoria_profissional').size()
taxa_cat = (problemas_cat / total_cat * 100).sort_values(ascending=False)

fig = go.Figure(data=[go.Bar(
    y=taxa_cat.index,
    x=taxa_cat.values,
    orientation='h',
    marker=dict(color=COLORS['primary']),
    text=[f"{v:.1f}%" for v in taxa_cat.values],
    textposition='outside'
)])

fig.update_layout(
    title='Taxa de Absentismo por Categoria Profissional (Top 10)',
    xaxis_title='Taxa (%)',
    yaxis_title='Categoria',
    height=500
)

fig.show()

print(f'\n📊 Variação entre categorias:')
print(f'  Maior taxa: {taxa_cat.index[0]} ({taxa_cat.values[0]:.1f}%)')
print(f'  Menor taxa: {taxa_cat.index[-1]} ({taxa_cat.values[-1]:.1f}%)')
print(f'  Diferença: {taxa_cat.values[0] - taxa_cat.values[-1]:.1f} pontos percentuais')

In [ ]:
# Top 15 operações por taxa de absentismo
top_ops = df['operacao'].value_counts().head(15).index
df_top_ops = df[df['operacao'].isin(top_ops)]

problemas_ops = df_top_ops[(df_top_ops['nivel_1'] == 'Ausência') | (df_top_ops['nivel_4'] == 'Atraso')].groupby('operacao').size()
total_ops = df_top_ops.groupby('operacao').size()
taxa_ops = (problemas_ops / total_ops * 100).sort_values(ascending=False)

fig = go.Figure(data=[go.Bar(
    y=taxa_ops.index,
    x=taxa_ops.values,
    orientation='h',
    marker=dict(color=[
        COLORS['danger'] if v > 7 else COLORS['warning'] if v > 5 else COLORS['success']
        for v in taxa_ops.values
    ]),
    text=[f"{v:.1f}%" for v in taxa_ops.values],
    textposition='outside'
)])

fig.update_layout(
    title='Taxa de Absentismo por Operação (Top 15)',
    xaxis_title='Taxa (%)',
    yaxis_title='Operação',
    height=600
)

fig.show()

---

# SEÇÃO 6: Red Flags e Abusos Óbvios 🚨

## Pergunta 10: Há evidências de abuso no uso de códigos específicos?

Vamos investigar padrões impossíveis ou altamente suspeitos a nível do código detalhado (Nível 1).

### Red Flag 1: Exames Escolares aos Fins de Semana???

In [ ]:
# Procurar ExameEscolar aos fins de semana
exames_fds = df[(df['segmento_processado_codigo'] == 'ExameEscolar') & 
                (df['dia_semana'].isin([5, 6]))].copy()

print('='*80)
print('🚨 RED FLAG 1: EXAMES ESCOLARES AOS FINS DE SEMANA')
print('='*80)

if len(exames_fds) > 0:
    print(f'\n❌ ENCONTRADOS {len(exames_fds)} registros de "ExameEscolar" aos sábados/domingos!')
    print(f'\nColaboradores envolvidos: {exames_fds["login_colaborador"].nunique()}')
    print(f'\nTop colaboradores com este padrão:')
    top_exames_fds = exames_fds['nome_colaborador'].value_counts().head(10)
    for nome, count in top_exames_fds.items():
        print(f'  • {nome}: {count} exames aos FDS')
    print(f'\n💡 RECOMENDAÇÃO: Investigar estes casos - escolas não abrem aos fins de semana!')
else:
    print(f'\n✅ Nenhum exame escolar registado aos fins de semana.')

print('='*80)

### Red Flag 2: Baixas Médicas Sempre Adjacentes a Fins de Semana

In [ ]:
# Baixas médicas às segundas ou sextas (suspeito de extensão de FDS)
baixas = df[df['segmento_processado_codigo'].isin(['BM', 'BMP'])].copy()
baixas_seg_sex = baixas[baixas['dia_semana'].isin([0, 4])]

print('='*80)
print('🚨 RED FLAG 2: BAIXAS MÉDICAS ÀS SEGUNDAS/SEXTAS')
print('='*80)

total_baixas = len(baixas)
baixas_suspeitas = len(baixas_seg_sex)
pct_suspeito = baixas_suspeitas / total_baixas * 100

print(f'\nTotal de dias em baixa médica: {total_baixas:,}')
print(f'Baixas às segundas ou sextas: {baixas_suspeitas:,} ({pct_suspeito:.1f}%)')

# Se fosse aleatório, esperaríamos ~28.6% (2 dias em 7)
esperado_pct = 28.6
if pct_suspeito > esperado_pct + 5:
    print(f'\n⚠️ ATENÇÃO: Percentagem acima do esperado (esperado ~{esperado_pct:.1f}%)')
    print(f'   Diferença: +{pct_suspeito - esperado_pct:.1f} pontos percentuais')

# Top colaboradores com baixas às seg/sex
print(f'\nTop 10 colaboradores com mais baixas médicas às segundas/sextas:')
top_baixas_fds = baixas_seg_sex.groupby(['login_colaborador', 'nome_colaborador']).size().sort_values(ascending=False).head(10)
for (login, nome), count in top_baixas_fds.items():
    print(f'  • {nome} (ID: {login}): {count} baixas às seg/sex')

print('='*80)

### Red Flag 3: Colaboradores com Faltas Injustificadas Recorrentes

In [ ]:
# Apenas colaboradores ativos
df_ativos = df[df['is_ativo'] == 1]

# Faltas Injustificadas (usar nivel_2)
faltas_inj = df_ativos[df_ativos['nivel_2'] == 'Ausência Injustificada']

print('='*80)
print('🚨 RED FLAG 3: FALTAS INJUSTIFICADAS RECORRENTES (ATIVOS)')
print('='*80)

# Contar por colaborador
faltas_por_colab = faltas_inj.groupby(['login_colaborador', 'nome_colaborador', 'operacao']).size().sort_values(ascending=False)

# Casos graves (>=10 faltas injustificadas)
casos_graves = faltas_por_colab[faltas_por_colab >= 10]

print(f'\nTotal de colaboradores ativos com faltas injustificadas: {len(faltas_por_colab)}')
print(f'Casos graves (≥10 faltas injustificadas): {len(casos_graves)}')

if len(casos_graves) > 0:
    print(f'\n🔴 TOP 20 CASOS GRAVES:')
    for (login, nome, op), count in casos_graves.head(20).items():
        print(f'  • {nome} ({op}): {count} faltas injustificadas')
else:
    print(f'\n✅ Nenhum caso com ≥10 faltas injustificadas')

print('='*80)

### Red Flag 4: Padrão "Segunda ou Sexta"

In [ ]:
# Colaboradores que faltam MUITO às segundas ou sextas
# Usar nivel_1 == 'Ausência' para capturar todas as ausências (justificadas e injustificadas)
faltas_total = df_ativos[df_ativos['nivel_1'] == 'Ausência']

# Por colaborador
faltas_seg = faltas_total[faltas_total['dia_semana'] == 0].groupby('login_colaborador').size()
faltas_sex = faltas_total[faltas_total['dia_semana'] == 4].groupby('login_colaborador').size()
faltas_total_count = faltas_total.groupby('login_colaborador').size()

# Calcular %
pct_seg = (faltas_seg / faltas_total_count * 100).fillna(0)
pct_sex = (faltas_sex / faltas_total_count * 100).fillna(0)

# Filtrar suspeitos (>=40% às segundas OU sextas, mínimo 5 faltas)
suspeitos_seg = pct_seg[(pct_seg >= 40) & (faltas_total_count >= 5)].sort_values(ascending=False)
suspeitos_sex = pct_sex[(pct_sex >= 40) & (faltas_total_count >= 5)].sort_values(ascending=False)

print('='*80)
print('🚨 RED FLAG 4: PADRÃO "EXTENSÃO DE FIM DE SEMANA"')
print('='*80)

print(f'\nColaboradores com ≥40% de faltas às SEGUNDAS (mín. 5 faltas): {len(suspeitos_seg)}')
if len(suspeitos_seg) > 0:
    print(f'\nTop 10:')
    for login in suspeitos_seg.head(10).index:
        nome = df_ativos[df_ativos['login_colaborador'] == login]['nome_colaborador'].iloc[0]
        total = faltas_total_count[login]
        pct = suspeitos_seg[login]
        print(f'  • {nome}: {pct:.0f}% das faltas às segundas ({faltas_seg.get(login, 0)}/{total} faltas)')

print(f'\n\nColaboradores com ≥40% de faltas às SEXTAS (mín. 5 faltas): {len(suspeitos_sex)}')
if len(suspeitos_sex) > 0:
    print(f'\nTop 10:')
    for login in suspeitos_sex.head(10).index:
        nome = df_ativos[df_ativos['login_colaborador'] == login]['nome_colaborador'].iloc[0]
        total = faltas_total_count[login]
        pct = suspeitos_sex[login]
        print(f'  • {nome}: {pct:.0f}% das faltas às sextas ({faltas_sex.get(login, 0)}/{total} faltas)')

print('='*80)

---

# SEÇÃO 7: Conclusões e Recomendações

## Sumário Executivo

In [ ]:
print('='*90)
print('📊 SUMÁRIO EXECUTIVO - ANÁLISE DE ABSENTISMO')
print('='*90)

print('\n1️⃣ PANORAMA GERAL')
print(f'   • Período: {periodo_inicio.strftime("%d/%m/%Y")} a {periodo_fim.strftime("%d/%m/%Y")} ({dias_calendario} dias)')
print(f'   • Dias-colaborador analisados: {len(df):,}')
print(f'   • Colaboradores ativos: {colaboradores_ativos:,}')

# Estatísticas por nível
taxa_ausencia_n1 = (df[df['nivel_1'] == 'Ausência'].shape[0] / len(df) * 100)
taxa_atrasos = (df[df['nivel_4'] == 'Atraso'].shape[0] / len(df) * 100)
taxa_total_problemas = (len(df_problemas) / len(df) * 100)

print(f'   • Taxa de Ausência (Nível 1): {taxa_ausencia_n1:.2f}%')
print(f'   • Taxa de Atrasos: {taxa_atrasos:.2f}%')
print(f'   • Taxa Total de Problemas (ausências + atrasos): {taxa_total_problemas:.2f}%')

print('\n2️⃣ PADRÕES TEMPORAIS IDENTIFICADOS')
# Calcular os insights principais
seg_count = df_problemas[df_problemas['dia_semana'] == 0].shape[0]
sex_count = df_problemas[df_problemas['dia_semana'] == 4].shape[0]
seg_pct = seg_count / len(df_problemas) * 100
sex_pct = sex_count / len(df_problemas) * 100
esperado_dia = 100 / 7  # Se fosse uniforme

print(f'   • Segundas-feiras: {seg_pct:.1f}% das ausências/atrasos (esperado ~{esperado_dia:.1f}%)')
print(f'   • Sextas-feiras: {sex_pct:.1f}% das ausências/atrasos (esperado ~{esperado_dia:.1f}%)')
print(f'   • Teste estatístico: Distribuição NÃO é aleatória (p < 0.001)')
if variacao_pct > 0:
    print(f'   • Tendência: 🔴 PIOROU {abs(variacao_pct):.1f}% vs início')
else:
    print(f'   • Tendência: 🟢 MELHOROU {abs(variacao_pct):.1f}% vs início')

print('\n3️⃣ SEGMENTAÇÃO')
print(f'   • Categoria com maior taxa: {taxa_cat.index[0]} ({taxa_cat.values[0]:.1f}%)')
print(f'   • Operação com maior taxa: {taxa_ops.index[0]} ({taxa_ops.values[0]:.1f}%)')
print(f'   • Variação significativa entre operações: {taxa_ops.values[0] - taxa_ops.values[-1]:.1f} p.p.')

print('\n4️⃣ RED FLAGS IDENTIFICADOS 🚨')
print(f'   • Exames escolares aos FDS: {len(exames_fds)} casos')
print(f'   • Baixas médicas seg/sex: {pct_suspeito:.1f}% (esperado ~28.6%)')
print(f'   • Casos graves (≥10 faltas injust.): {len(casos_graves)} colaboradores')
print(f'   • Padrão "Segunda": {len(suspeitos_seg)} colaboradores')
print(f'   • Padrão "Sexta": {len(suspeitos_sex)} colaboradores')

print('\n5️⃣ CLASSIFICAÇÃO HIERÁRQUICA')
print(f'   Nível 1 (Presença vs Ausência):')
for cat in dist_nivel1.index:
    pct = dist_nivel1[cat] / len(df) * 100
    print(f'     • {cat}: {pct:.2f}%')
print(f'   Nível 2 (4 categorias) - Top 3:')
for cat in dist_nivel2.index[::-1][:3]:
    pct = dist_nivel2[cat] / len(df) * 100
    print(f'     • {cat}: {pct:.2f}%')

print('\n' + '='*90)

## Principais Insights e Recomendações

In [ ]:
print('='*90)
print('💡 PRINCIPAIS INSIGHTS')
print('='*90)

insights = [
    '1. PADRÃO NÃO ALEATÓRIO: Teste estatístico confirma que faltas não são aleatórias.',
    '   Há clara concentração às segundas e sextas-feiras (extensão de fim de semana).',
    '',
    '2. CASOS FLAGRANTES: Identificados casos de fraude evidente (exames escolares aos FDS).',
    '   Requer ação disciplinar imediata.',
    '',
    '3. COMPORTAMENTO ESTRATÉGICO: Grupos significativos de colaboradores demonstram',
    '   padrões consistentes de faltas em dias específicos.',
    '',
    '4. VARIAÇÃO POR OPERAÇÃO: Diferenças significativas entre operações sugerem',
    '   possíveis problemas de gestão ou cultura em áreas específicas.',
]

for insight in insights:
    print(insight)

print('\n' + '='*90)
print('🎯 RECOMENDAÇÕES PRIORITÁRIAS')
print('='*90)

recomendacoes = [
    '\nPRIORIDADE ALTA (Ação Imediata):',
    '  1. Investigar casos de fraude (exames escolares aos FDS)',
    '  2. Reuniões disciplinares com colaboradores ≥10 faltas injustificadas',
    '  3. Advertências formais para padrão "Segunda/Sexta" recorrente',
    '',
    'PRIORIDADE MÉDIA (Curto Prazo):',
    '  4. Implementar sistema de alertas automático para padrões suspeitos',
    '  5. Revisão de processos nas operações com taxa > 7%',
    '  6. Verificação rigorosa de baixas médicas às segundas/sextas',
    '',
    'PRIORIDADE NORMAL (Médio Prazo):',
    '  7. Programa de sensibilização sobre impacto do absentismo',
    '  8. Reconhecimento de colaboradores com baixo absentismo',
    '  9. Revisão trimestral de métricas por operação',
    '',
    'FERRAMENTAS:',
    '  • Dashboard de monitorização em tempo real',
    '  • Relatórios mensais por gestor',
    '  • Sistema de scoring de risco por colaborador',
]

for rec in recomendacoes:
    print(rec)

print('\n' + '='*90)
print('✅ ANÁLISE CONCLUÍDA')
print('='*90)

---

## Próximos Passos

1. **Apresentar** este relatório à direção
2. **Exportar** listas de colaboradores para ação (ver notebooks de suporte)
3. **Implementar** sistema de monitorização contínua
4. **Reavaliar** em 3 meses para medir impacto das ações

---

**Nota**: Este notebook foi criado para storytelling baseado em dados.
Para análises ad-hoc e consultas individuais, consultar notebooks complementares.